# 577. Employee Bonus

### Difficulty
**Easy**

---

## Problem Statement

Given the `Employee` and `Bonus` tables, write a **SQL query** to report the **name** and **bonus amount** of each employee whose bonus is **less than 1000**.

- If an `Employee` does not have a record in the `Bonus` table, their bonus is considered `null`, which is treated as less than 1000.

Return the result table in **any order**.

---

## Table Schema

### **Table: Employee**
| Column Name | Type    |
|-------------|---------|
| `empId`     | `int`   |
| `name`      | `varchar` |
| `supervisor`| `int`   |
| `salary`    | `int`   |

- `empId` is the **column with unique values** for this table.
- Each row represents:
  - An **employee's ID** (`empId`),
  - Their **name** (`name`),
  - Their **supervisor's ID** (`supervisor`), and
  - Their **salary** (`salary`).

---

### **Table: Bonus**
| Column Name | Type |
|-------------|------|
| `empId`     | `int` |
| `bonus`     | `int` |

- `empId` is the **column with unique values** for this table.
- `empId` is a **foreign key** referencing the `empId` in the `Employee` table.
- Each row contains:
  - An **employee's ID** (`empId`),
  - Their **bonus amount** (`bonus`).

---

## Example

### **Input**
#### **Employee table:**
| empId | name   | supervisor | salary |
|-------|--------|------------|--------|
| 3     | Brad   | `null`     | 4000   |
| 1     | John   | 3          | 1000   |
| 2     | Dan    | 3          | 2000   |
| 4     | Thomas | 3          | 4000   |

#### **Bonus table:**
| empId | bonus |
|-------|-------|
| 2     | 500   |
| 4     | 2000  |

---

### **Output**
| name   | bonus |
|--------|-------|
| Brad   | `null` |
| John   | `null` |
| Dan    | 500   |

---

### **Explanation**
- **Brad** and **John** have no entry in the `Bonus` table, so their bonus is `null`.  
  Since `null` is treated as less than 1000, they are included in the result.
- **Dan** has a bonus of `500`, which is less than 1000, so he is included.
- **Thomas** has a bonus of `2000`, which is greater than 1000, so he is excluded.

---

## **Hints**
1. If the `empId` in the `Employee` table has **no match** in the `Bonus` table, the bonus is considered `null`, and `null` is treated as less than 1000.
2. Use an **outer join** (e.g., `LEFT JOIN`) to handle employees without a record in the `Bonus` table.

---



# Solution

In [1]:
import pandas as pd

In [2]:
def employee_bonus(employee: pd.DataFrame, bonus: pd.DataFrame) -> pd.DataFrame:
    bonus_employee_merged = pd.merge(bonus, employee, how='right', on='empId')
    bonus_employee_modified = bonus_employee_merged[(bonus_employee_merged['bonus'] < 1000) | (bonus_employee_merged['bonus'].isnull())].copy()
    return bonus_employee_modified[['name','bonus']]

# Alternative Solutions

### **Alternative Pandas Solutions for Employee Bonus Problem**
Here are a few alternative solutions for this problem, along with their complexity analysis and comparison.

---

### **1️⃣ Using `pd.merge` with `how='left'`**
This is similar to your solution but explicitly uses a **`LEFT JOIN`** (more intuitive for this problem).

```python
def employee_bonus(employee: pd.DataFrame, bonus: pd.DataFrame) -> pd.DataFrame:
    bonus_employee_merged = pd.merge(employee, bonus, how='left', on='empId')
    bonus_employee_filtered = bonus_employee_merged[
        (bonus_employee_merged['bonus'] < 1000) | (bonus_employee_merged['bonus'].isnull())
    ].copy()
    return bonus_employee_filtered[['name', 'bonus']]
```

#### **Key Difference**
- `how='left'` makes it clear that `employee` is the base table.

#### **Complexity**
- **Time Complexity:** O(n + m) (same as your original solution).
- **Space Complexity:** O(n + m) (same as your original solution).

---

### **2️⃣ Using `employee.apply`**
Instead of merging, iterate over `employee` and look up `bonus` values for each `empId`.

```python
def employee_bonus(employee: pd.DataFrame, bonus: pd.DataFrame) -> pd.DataFrame:
    bonus_dict = bonus.set_index('empId')['bonus'].to_dict()
    employee['bonus'] = employee['empId'].map(bonus_dict)
    filtered = employee[(employee['bonus'] < 1000) | (employee['bonus'].isnull())].copy()
    return filtered[['name', 'bonus']]
```

#### **How It Works**
1. Convert the `bonus` DataFrame into a dictionary for faster lookups:
   ```python
   bonus_dict = bonus.set_index('empId')['bonus'].to_dict()
   ```
2. Map the `bonus` dictionary to `employee['empId']`:
   ```python
   employee['bonus'] = employee['empId'].map(bonus_dict)
   ```
3. Filter rows where `bonus < 1000` or `bonus` is `null`.

#### **Complexity**
- **Time Complexity:**
  - Creating `bonus_dict`: O(m).
  - Mapping `bonus` to `employee`: O(n).
  - Filtering: O(n).
  - **Total:** O(n + m).
- **Space Complexity:** O(m) (for the dictionary).

#### **Pros**
- Simple and avoids a full merge.
- Efficient for small `bonus` tables (`m` is small).

#### **Cons**
- Not as intuitive as `merge` for larger datasets or complex joins.

---

### **3️⃣ Using `query` for Filtering**
This is a variation of the `merge` solution that uses `query` for more concise filtering.

```python
def employee_bonus(employee: pd.DataFrame, bonus: pd.DataFrame) -> pd.DataFrame:
    bonus_employee_merged = pd.merge(employee, bonus, how='left', on='empId')
    return bonus_employee_merged.query('bonus < 1000 or bonus.isnull()')[['name', 'bonus']]
```

#### **How It Works**
- Merges `employee` and `bonus` using `how='left'`.
- Filters rows using `query`, which can improve readability.

#### **Complexity**
- **Time Complexity:** O(n + m) (same as `merge` solution).
- **Space Complexity:** O(n + m).

#### **Pros**
- More concise filtering using `query`.
- Similar performance to the original solution.

---

### **4️⃣ Using `join` Instead of `merge`**
If `bonus` is indexed by `empId`, you can use `join` for a simpler approach.

```python
def employee_bonus(employee: pd.DataFrame, bonus: pd.DataFrame) -> pd.DataFrame:
    bonus = bonus.set_index('empId')
    employee = employee.set_index('empId')
    bonus_employee_joined = employee.join(bonus, how='left').reset_index()
    filtered = bonus_employee_joined[
        (bonus_employee_joined['bonus'] < 1000) | (bonus_employee_joined['bonus'].isnull())
    ]
    return filtered[['name', 'bonus']]
```

#### **How It Works**
- Sets both `employee` and `bonus` to be indexed by `empId`.
- Performs a `join` on the indices.
- Filters rows with a condition similar to the `merge` approach.

#### **Complexity**
- **Time Complexity:**
  - Setting indices: O(n + m).
  - Joining: O(n + m).
  - Filtering: O(n).
  - **Total:** O(n + m).
- **Space Complexity:** O(n + m) (space for indices).

#### **Pros**
- Efficient if `bonus` is already indexed by `empId`.
- Avoids unnecessary columns being merged.

#### **Cons**
- Requires resetting the index if the original DataFrame index matters.

---

### **Comparison of Solutions**

| **Approach**              | **Time Complexity** | **Space Complexity** | **Best For**                       |
|----------------------------|---------------------|-----------------------|-------------------------------------|
| **Merge (`how='left'`)**   | **O(n + m)**        | **O(n + m)**          | General use, intuitive and scalable. |
| **Merge (`how='right'`)**  | **O(n + m)**        | **O(n + m)**          | Equivalent to `LEFT JOIN`.          |
| **Mapping (`apply`)**      | **O(n + m)**        | **O(m)**              | Small `bonus` tables, simpler logic. |
| **`query`** for filtering  | **O(n + m)**        | **O(n + m)**          | More concise filtering, readable.   |
| **`join`** with indices    | **O(n + m)**        | **O(n + m)**          | Pre-indexed tables, efficient joins.|

---
